In [23]:
import pandas as pd
import mysql.connector
import configparser


<h2>RESET BASE DE DATOS</h2>

In [24]:
# Leer el archivo de configuración
config = configparser.ConfigParser()
config.read('config.ini')

# Obtener los valores del archivo de configuración
host = config['mysql']['host']
user = config['mysql']['user']
password = config['mysql']['password']

conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password
)

cursor = conn.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS red_electrica")

conn.commit()

conn.close()

print("Base de datos red_electrica eliminada con éxito si existía.")

Base de datos red_electrica eliminada con éxito si existía.


<h2>TABLA DE BALANCE</h2>

In [25]:
# Conectar a la base de datos
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password
)

# Crear cursor
cursor = conn.cursor()

# Usar la base de datos
cursor.execute("USE red_electrica")

# Crear tabla
cursor.execute("""
    CREATE TABLE IF NOT EXISTS balance (
        id INT AUTO_INCREMENT PRIMARY KEY,
        nombre VARCHAR(50) NOT NULL,
        tipo_energia VARCHAR(50) NOT NULL,
        valores FLOAT NOT NULL,
        porcentaje FLOAT NOT NULL,
        fecha_actualizacion DATE NOT NULL
    )
""")

# Confirmar cambios
conn.commit()

# Cerrar conexión
cursor.close()
conn.close()

print("Tabla 'balance' creada con éxito.")


Tabla 'balance' creada con éxito.


<h3>CARGAR BALANCE</h3>

In [26]:
import pandas as pd
import mysql.connector

# Cargar el CSV con la interpretación correcta de las fechas
df = pd.read_csv('../Obtencion datos/balance_electrico.csv', sep=',', 
                 parse_dates=['Fecha actualización'], dayfirst=True)

# Crear una columna 'id' usando el índice del DataFrame
df['id'] = df.index + 1  # Genera un id único basado en el índice

# Renombrar columnas si es necesario
df.rename(columns={'tipo de energía': 'tipo_energia'}, inplace=True)

# Conectar a la base de datos
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database='red_electrica'
)

# Crear cursor
cursor = conn.cursor()

# Insertar o actualizar datos en la tabla usando el id
for index, row in df.iterrows():
    cursor.execute(""" 
        INSERT INTO balance (id, nombre, tipo_energia, valores, porcentaje, fecha_actualizacion)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
        nombre = VALUES(nombre),
        tipo_energia = VALUES(tipo_energia),
        valores = VALUES(valores),
        porcentaje = VALUES(porcentaje),
        fecha_actualizacion = VALUES(fecha_actualizacion)
    """, (row['id'], row['nombre'], row['tipo_energia'], row['Valores'], row['Porcentaje'], row['Fecha actualización']))

# Confirmar los cambios
conn.commit()

# Cerrar conexión
cursor.close()
conn.close()

print("Datos insertados o actualizados en la tabla 'balance' con éxito.")


Datos insertados o actualizados en la tabla 'balance' con éxito.


<h2>TABLA DEMANDA</h2>

In [27]:
# Conectar a la base de datos
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password
)

# Crear cursor
cursor = conn.cursor()

# Usar la base de datos
cursor.execute("USE red_electrica")

# Crear tabla
cursor.execute("""
    CREATE TABLE IF NOT EXISTS demanda (
        id INT AUTO_INCREMENT PRIMARY KEY,
        fecha DATE NOT NULL,
        valor FLOAT NOT NULL
    )
""")

# Confirmar cambios
conn.commit()

# Cerrar conexión
cursor.close()
conn.close()

print("Tabla 'demanda' creada con éxito.")


Tabla 'demanda' creada con éxito.


<h3>CARGAR DEMANDA</h3>

In [28]:
import pandas as pd
import mysql.connector
import configparser

# Leer el archivo de configuración
config = configparser.ConfigParser()
config.read('config.ini')

# Obtener los valores del archivo de configuración
host = config['mysql']['host']
user = config['mysql']['user']
password = config['mysql']['password']

# Cargar el CSV y mostrar las primeras filas para depuración
try:
    df_raw = pd.read_csv('../Obtencion datos/demanda_evolucion.csv', sep=',')
except FileNotFoundError as e:
    print(f"Error: {e}")
    exit()

# Asegurarse de que la columna 'Fecha actualización' existe antes de parsearla
if 'Fecha actualización' not in df_raw.columns:
    print("Error: La columna 'Fecha actualización' no existe en el CSV.")
    exit()

# Cargar el CSV con el nombre correcto de la columna para la fecha
df = pd.read_csv('../Obtencion datos/demanda_evolucion.csv', sep=',', 
                 parse_dates=['Fecha actualización'], 
                 dayfirst=True)

df.rename(columns={'Fecha actualización': 'fecha', 'Energia_consumida': 'valor'}, inplace=True)


# Crear una columna 'id' usando el índice del DataFrame
df['id'] = df.index + 1 


try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database='red_electrica'
    )

    with conn.cursor() as cursor:
        # Insertar o actualizar datos en la tabla usando el id como clave única
        for index, row in df.iterrows():
            cursor.execute(""" 
                INSERT INTO demanda (id, fecha, valor)
                VALUES (%s, %s, %s)
                ON DUPLICATE KEY UPDATE
                fecha = VALUES(fecha),
                valor = VALUES(valor)
            """, (row['id'], row['fecha'], row['valor']))

    # Confirmar los cambios
    conn.commit()

except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    if conn.is_connected():
        conn.close()

print("Datos insertados o actualizados en la tabla 'demanda' con éxito.")


Datos insertados o actualizados en la tabla 'demanda' con éxito.


<h2>TABLA GENERACIÓN</h2>

In [29]:
import mysql.connector
import configparser

# Leer el archivo de configuración
config = configparser.ConfigParser()
config.read('config.ini')

# Obtener los valores del archivo de configuración
host = config['mysql']['host']
user = config['mysql']['user']
password = config['mysql']['password']

# Conectar a la base de datos
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password
)

# Crear cursor
cursor = conn.cursor()

# Usar la base de datos
cursor.execute("USE red_electrica")

# Crear tabla
cursor.execute("""
    CREATE TABLE IF NOT EXISTS generacion (
        id INT AUTO_INCREMENT PRIMARY KEY,
        nombre VARCHAR(50) NOT NULL,
        tipo_energia VARCHAR(50) NOT NULL,
        valores FLOAT NOT NULL,
        porcentaje FLOAT NOT NULL,
        fecha_actualizacion DATE NOT NULL
    )
""")

# Confirmar cambios
conn.commit()

# Cerrar conexión
cursor.close()
conn.close()

print("Tabla 'generacion' creada con éxito.")


Tabla 'generacion' creada con éxito.


<h3>CARGAR GENERACION</h3>

In [30]:
import pandas as pd
import mysql.connector
import configparser

# Leer el archivo de configuración
config = configparser.ConfigParser()
config.read('config.ini')

# Obtener los valores del archivo de configuración
host = config['mysql']['host']
user = config['mysql']['user']
password = config['mysql']['password']

# Cargar el CSV sin el parser de fecha
df = pd.read_csv('../Obtencion datos/generacion_estructura.csv', sep=',')

# Convertir la columna 'Fecha actualización' a formato de fecha
df['Fecha actualización'] = pd.to_datetime(df['Fecha actualización'], format='%d/%m/%Y')

# Renombrar las columnas para que coincidan con la estructura de la tabla
df.rename(columns={
    'tipo de energía': 'tipo_energia', 
    'Valores': 'valores', 
    'Porcentaje': 'porcentaje', 
    'Fecha actualización': 'fecha_actualizacion'
}, inplace=True)

# Crear una columna 'id' usando el índice del DataFrame
df['id'] = df.index + 1  # Genera un id único basado en el índice

# Conectar a la base de datos
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database='red_electrica'
)

# Crear cursor
cursor = conn.cursor()

# Insertar o actualizar datos en la tabla usando el id como clave única
for index, row in df.iterrows():
    cursor.execute("""
        INSERT INTO generacion (id, nombre, tipo_energia, valores, porcentaje, fecha_actualizacion)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
        nombre = VALUES(nombre),
        tipo_energia = VALUES(tipo_energia),
        valores = VALUES(valores),
        porcentaje = VALUES(porcentaje),
        fecha_actualizacion = VALUES(fecha_actualizacion)
    """, (row['id'], row['nombre'], row['tipo_energia'], row['valores'], row['porcentaje'], row['fecha_actualizacion']))

# Confirmar los cambios
conn.commit()

# Cerrar conexión
cursor.close()
conn.close()

print("Datos insertados o actualizados en la tabla 'generacion' con éxito.")


Datos insertados o actualizados en la tabla 'generacion' con éxito.


<h2>TABLA INTERCAMBIO</h2>

In [31]:
import mysql.connector
import configparser

# Leer el archivo de configuración
config = configparser.ConfigParser()
config.read('config.ini')

# Obtener los valores del archivo de configuración
host = config['mysql']['host']
user = config['mysql']['user']
password = config['mysql']['password']

# Conectar a la base de datos
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password
)

# Crear cursor
cursor = conn.cursor()

# Usar la base de datos
cursor.execute("USE red_electrica")

# Crear tabla
cursor.execute("""
    CREATE TABLE IF NOT EXISTS intercambio (
        id INT AUTO_INCREMENT PRIMARY KEY,
        nombre VARCHAR(50) NOT NULL,
        tipo_intercambio VARCHAR(50) NOT NULL,
        valores FLOAT NOT NULL,
        porcentaje FLOAT NOT NULL,
        fecha_actualizacion DATE NOT NULL
    )
""")

# Confirmar cambios
conn.commit()

# Cerrar conexión
cursor.close()
conn.close()

print("Tabla 'intercambio' creada con éxito.")


Tabla 'intercambio' creada con éxito.


<h3>CARGAR INTERCAMBIO</h3>

In [32]:
import pandas as pd
import mysql.connector
import configparser

# Leer el archivo de configuración
config = configparser.ConfigParser()
config.read('config.ini')

# Obtener los valores del archivo de configuración
host = config['mysql']['host']
user = config['mysql']['user']
password = config['mysql']['password']

# Definir un parser para las fechas
date_parser = lambda x: pd.to_datetime(x, format='%d/%m/%Y')  # Cambia el formato si es necesario

# Cargar el CSV
df = pd.read_csv('../Obtencion datos/intercambio_electrico.csv', sep=',', parse_dates=['Fecha actualización'], date_parser=date_parser)

# Renombrar las columnas para que coincidan con la estructura de la tabla
df.rename(columns={
    'tipo de intercambio': 'tipo_intercambio', 
    'Valores': 'valores', 
    'Porcentaje': 'porcentaje', 
    'Fecha actualización': 'fecha_actualizacion'
}, inplace=True)

# Crear una columna 'id' usando el índice del DataFrame
df['id'] = df.index + 1  # Genera un id único basado en el índice

# Conectar a la base de datos
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database='red_electrica'
)

# Crear cursor
cursor = conn.cursor()

# Insertar o actualizar datos en la tabla usando el id como clave única
for index, row in df.iterrows():
    cursor.execute("""
        INSERT INTO intercambio (id, nombre, tipo_intercambio, valores, porcentaje, fecha_actualizacion)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
        nombre = VALUES(nombre),
        tipo_intercambio = VALUES(tipo_intercambio),
        valores = VALUES(valores),
        porcentaje = VALUES(porcentaje),
        fecha_actualizacion = VALUES(fecha_actualizacion)
    """, (row['id'], row['nombre'], row['tipo_intercambio'], row['valores'], row['porcentaje'], row['fecha_actualizacion']))

# Confirmar los cambios
conn.commit()

# Cerrar conexión
cursor.close()
conn.close()

print("Datos insertados o actualizados en la tabla 'intercambio' con éxito.")


C:\Users\SAPASHO\AppData\Local\Temp\ipykernel_9336\3277607746.py:18: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv('../Obtencion datos/intercambio_electrico.csv', sep=',', parse_dates=['Fecha actualización'], date_parser=date_parser)


Datos insertados o actualizados en la tabla 'intercambio' con éxito.
